In [2]:
# 引入TensorFlow
import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [3]:
# 1 导入数据集
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
# 2 数据处理-分成batch，混淆数据
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [13]:
# 3.1使用面向对象的方式定义一个模型
# 对于多分类问题，一般定义有多个分类的输出。例如10分类，则结果有十个。并且可以将结果添加softmax层，输出每一个的概率。然后使用交叉信息熵，计算这十个输出的结果，与目标结果的交叉损失。（目标结果，本质上应该是十个元素的向量，onehot编码的向量。
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
  # 3.2 进行正向传播===pytorch.forward   
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()

In [7]:
# 3.3选择损失函数和3.5定义优化器
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [8]:
# 4 选择评估指标
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
# 3 定义梯度下降的步骤
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [10]:
# 4 测试模型
@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # 在下一个epoch开始时，重置评估指标
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

Epoch 1, Loss: 0.136604443192482, Accuracy: 95.85000610351562, Test Loss: 0.07532097399234772, Test Accuracy: 97.58999633789062
Epoch 2, Loss: 0.04355545714497566, Accuracy: 98.5999984741211, Test Loss: 0.04925746098160744, Test Accuracy: 98.32999420166016
Epoch 3, Loss: 0.023197738453745842, Accuracy: 99.24666595458984, Test Loss: 0.05581401661038399, Test Accuracy: 98.30999755859375
Epoch 4, Loss: 0.013380118645727634, Accuracy: 99.54000091552734, Test Loss: 0.057846762239933014, Test Accuracy: 98.45999908447266
Epoch 5, Loss: 0.010646246373653412, Accuracy: 99.625, Test Loss: 0.07796232402324677, Test Accuracy: 98.1199951171875


In [12]:
print(model.summary())

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2769024   
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________
None
